# Lab | Simple LLM App with LCEL

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```

In [6]:
!pip install -U langchain langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [7]:
import os
os.environ["OPENAI_API_KEY"] = "my key"


In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)

# Simple test:
response = llm.invoke("Give me a simple summary about artificial intelligence")
print(response.content)


Artificial intelligence (AI) is a branch of computer science that focuses on creating machines that can perform tasks that typically require human intelligence, such as speech recognition, decision making, and problem solving. AI systems can learn from experience, adapt to new information, and make decisions based on data analysis. AI technology is being used in a wide range of industries, from healthcare and finance to transportation and entertainment.


Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Define the model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # ← This is the correct model name
    temperature=0.7
)

# Create the message sequence
messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="How are you today?")
]

# Call the model with the messages
response = llm.invoke(messages)

# Print the model's response
print(response.content)


Come stai oggi?


If we've enable LangSmith, we can see that this run is logged to LangSmith, and can see the [LangSmith trace](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)

## OutputParsers

Notice that the response from the model is an `AIMessage`. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [17]:
result = llm.invoke(messages)


In [18]:
parser.invoke(result)

'Come stai oggi?'

More commonly, we can "chain" the model with this output parser. This means this output parser will get called everytime in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the `|` operator. The `|` operator is used in LangChain to combine two elements together.

In [22]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Initialize the variable 'chain' as None first
chain = None

# Build the chain later
prompt = PromptTemplate.from_template("Translate to Spanish: {text}")
chain = LLMChain(llm=llm, prompt=prompt)

# Now you can use the chain
result = chain.run("Good evening")
print(result)


<ipython-input-22-385c8f23beac>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-22-385c8f23beac>:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run("Good evening")


Buenas tardes


In [23]:
chain.invoke(messages)

{'text': "[SystemMessage(content='Traduce lo siguiente del inglés al italiano', additional_kwargs={}, response_metadata={}), HumanMessage(content='¿Cómo estás hoy?', additional_kwargs={}, response_metadata={})]"}

If we now look at LangSmith, we can see that the chain has two steps: first the language model is called, then the result of that is passed to the output parser. We can see the [LangSmith trace]( https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r)

## Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a PromptTemplate here. It will take in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate

In [24]:
from langchain_core.prompts import ChatPromptTemplate

First, let's create a string that we will format to be the system message:

In [25]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the `system_template` as well as a simpler template for where the put the text

In [26]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [27]:
result = prompt_template.invoke({"language": "italian", "text": None})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='None', additional_kwargs={}, response_metadata={})])

We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [28]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='None', additional_kwargs={}, response_metadata={})]

## Chaining together components with LCEL

We can now combine this with the model and the output parser from above using the pipe (`|`) operator:

In [32]:
chain = None

In [35]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the LLM model (e.g., OpenAI's GPT-3.5 or GPT-4)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)  # Make sure to use the correct model name

# Build the prompt template
prompt = PromptTemplate.from_template("Translate to {language}: {text}")

# Create the chain with the LLM model and the prompt template
chain = LLMChain(llm=llm, prompt=prompt)
ؤ
# Send the data to the chain with an actual text to translate
result = chain.invoke({"language": "italian", "text": "Hello, how are you?"})

# Print the result
print(result)


{'language': 'italian', 'text': 'Ciao, come stai?'}


This is a simple example of using [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language-lcel) to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

If we take a look at the LangSmith trace, we can see all three components show up in the [LangSmith trace](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r).

## Serving with LangServe

Now that we've built an application, we need to serve it. That's where LangServe comes in.
LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:
```bash
pip install "langserve[all]"
```

### Server

To create a server for our application we'll make a `serve.py` file. This will contain our logic for serving our application. It consists of three things:
1. The definition of our chain that we just built above
2. Our FastAPI app
3. A definition of a route from which to serve the chain, which is done with `langserve.add_routes`


```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

And that's it! If we execute this file:
```bash
python serve.py
```
we should see our chain being served at [http://localhost:8000](http://localhost:8000).

### Playground

Every LangServe service comes with a simple [built-in UI](https://github.com/langchain-ai/langserve/blob/main/README.md#playground) for configuring and invoking the application with streaming output and visibility into intermediate steps.
Head to [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/) to try it out! Pass in the same inputs as before - `{"language": "italian", "text": "hi"}` - and it should respond same as before.

### Client

Now let's set up a client for programmatically interacting with our service. We can easily do this with the `[langserve.RemoteRunnable](/docs/langserve/#client)`.
Using this, we can interact with the served chain as if it were running client-side.

In [51]:
!pip install fastapi langchain langserve uvicorn

In [43]:
!pip install "langserve[all]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [40]:
!pip install fastapi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00


In [38]:
!pip install langserve


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00


In [61]:
# Import RemoteRunnable from langserve to interact with a remote LangChain server
from langserve import RemoteRunnable

# Initialize a connection to the remote chain running at localhost:8000
# Note: Make sure your server is actually running at this address!
remote_chain = RemoteRunnable("http://localhost:8000/chain/")

# Prepare the input for our translation request
# We want to translate to Italian, and our text is "Hello, how are you?"
translation_request = {
    "language": "italian",  # Target language
    "text": "Hello, how are you?"  # Text to translate
}

# Try to invoke the remote chain with our request
try:
    # Send the request to the server and get the response
    result = remote_chain.invoke(translation_request)

    # Print the translated result
    print("Translation result:", result)

# Handle potential connection errors
except Exception as e:
    print("Failed to connect to server. Error:", e)
    print("\nTroubleshooting tips:")
    print("1. Make sure the server is running (execute 'python serve.py')")
    print("2. Verify the server is using host='0.0.0.0' not 'localhost'")
    print("3. Check if port 8000 is available or try a different port")
    print("4. If using Colab, you may need ngrok for external access")

Translation result: content='Ciao, come stai?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BOo6gGnMpv4qsMBNkKkAA3wm5b2oP', 'finish_reason': 'stop', 'logprobs': None} id='run-fd28f320-c545-457f-8c3e-879331da8a7b-0' usage_metadata={'input_tokens': 17, 'output_tokens': 8, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [55]:
!pip install fastapi langchain langserve uvicorn pyngrok

In [59]:
%%writefile serve.py
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langserve import add_routes

app = FastAPI()

prompt = ChatPromptTemplate.from_template("Translate to {language}: {text}")
model = ChatOpenAI()
chain = prompt | model

add_routes(app, chain, path="/chain")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

Overwriting serve.py


In [58]:
# 1. Install required libraries
!pip install fastapi langchain langserve uvicorn pyngrok

# 2. Import necessary modules
from pyngrok import ngrok  # For creating public tunnels
import subprocess  # To run shell commands
import threading  # For running the server in background

# 3. Add your ngrok authentication token (replace with your actual token)
NGROK_TOKEN = "my key"
ngrok.set_auth_token(NGROK_TOKEN)

# 4. Function to run the server in background
def run_server():
    """Starts the FastAPI server using subprocess"""
    subprocess.run(["python", "serve.py"])

# Start the server in a separate thread
thread = threading.Thread(target=run_server)
thread.start()

# 5. Create ngrok tunnel
try:
    public_url = ngrok.connect(8000)  # Connects to port 8000
    print("✅ Server started successfully!")
    print("🔗 Public URL:", public_url)
    print("\nYou can test it using:")
    print(f"- Interactive playground: {public_url}/chain/playground/")
    print(f"- Programmatic access: {public_url}/chain/invoke")
except Exception as e:
    print("❌ Failed to start due to:", e)

✅ Server started successfully!
🔗 Public URL: NgrokTunnel: "https://7e37-34-125-248-81.ngrok-free.app" -> "http://localhost:8000"

You can test it using:
- Interactive playground: NgrokTunnel: "https://7e37-34-125-248-81.ngrok-free.app" -> "http://localhost:8000"/chain/playground/
- Programmatic access: NgrokTunnel: "https://7e37-34-125-248-81.ngrok-free.app" -> "http://localhost:8000"/chain/invoke


To learn more about the many other features of LangServe [head here](/docs/langserve).

## Conclusion

That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to parse their outputs, how to create a prompt template, chaining them with LCEL, how to get great observability into chains you create with LangSmith, and how to deploy them with LangServe.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!

For further reading on the core concepts of LangChain, we've got detailed [Conceptual Guides](/docs/concepts).

If you have more specific questions on these concepts, check out the following sections of the how-to guides:

- [LangChain Expression Language (LCEL)](/docs/how_to/#langchain-expression-language-lcel)
- [Prompt templates](/docs/how_to/#prompt-templates)
- [Chat models](/docs/how_to/#chat-models)
- [Output parsers](/docs/how_to/#output-parsers)
- [LangServe](/docs/langserve/)

And the LangSmith docs:

- [LangSmith](https://docs.smith.langchain.com)